- We import necessary libraries

In [18]:
import pandas as pd
import numpy as np

- Read the dataset to be processed

In [19]:
df = pd.read_csv("D:\Project\Job-Market-Intelligence\data\\raw\data_jobs_cleaned.csv")


In [20]:
df.head(5)

,job_title_short,job_location,job_via,job_schedule_type,job_work_from_home,search_location,job_posted_date,job_no_degree_mention,job_health_insurance,job_country,salary_year_avg,company_name,job_skills,job_type_skills
0,Data Scientist,"San José Province, San José, Costa Rica",ai-jobs.net,Full-time,False,Costa Rica,2023-08-01 13:37:57,False,False,Costa Rica,109500.0,Netskope,"['gdpr', 'excel']","{'analyst_tools': ['excel'], 'libraries': ['gd..."
1,Data Engineer,"Arlington, VA",linkedin,Full-time,False,Sudan,2023-06-26 14:22:54,False,False,United Kingdom,140000.0,Intelletec,"['mongodb', 'mongodb', 'python', 'r', 'sql', '...","{'analyst_tools': ['tableau'], 'cloud': ['orac..."
2,Data Engineer,Anywhere,linkedin,Full-time,True,"Illinois, United States",2023-02-21 13:29:59,False,True,United States,120000.0,Apex Systems,"['sql', 'python']","{'programming': ['sql', 'python']}"
3,Data Scientist,"Mountain View, CA",linkedin,Full-time,False,"California, United States",2023-07-31 13:01:18,False,True,United States,228222.0,TikTok,"['sql', 'r', 'python', 'express']","{'programming': ['sql', 'r', 'python'], 'webfr..."
4,Data Analyst,Anywhere,get.it,Full-time,True,"Illinois, United States",2023-10-12 13:02:19,False,True,United States,89000.0,Get It Recruit - Transportation,"['python', 'r', 'alteryx', 'tableau']","{'analyst_tools': ['alteryx', 'tableau'], 'pro..."


- Drop the columns which have no proper weight for analysis
- Rename the columns for better understanding

In [21]:
df = df.drop(columns=["job_via",  "search_location"])
df = df.rename(columns={
    "job_title_short": "job_role",
    "job_country": "country",
    "job_work_from_home": "is_remote",
    "job_schedule_type": "schedule_type",
    "job_no_degree_mention": "no_degree_required",
    "job_health_insurance": "health_insurance",
    "job_posted_date": "posted_date"
})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19908 entries, 0 to 19907
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   job_role            19908 non-null  object 
 1   job_location        19908 non-null  object 
 2   schedule_type       19908 non-null  object 
 3   is_remote           19908 non-null  bool   
 4   posted_date         19908 non-null  object 
 5   no_degree_required  19908 non-null  bool   
 6   health_insurance    19908 non-null  bool   
 7   country             19908 non-null  object 
 8   salary_year_avg     19908 non-null  float64
 9   company_name        19908 non-null  object 
 10  job_skills          19908 non-null  object 
 11  job_type_skills     19908 non-null  object 
dtypes: bool(3), float64(1), object(8)
memory usage: 1.4+ MB


- Since this dataset contains many Data roles like "Data Scientist", "Data Engineer" and many more, it produces lot of noise and loses consistency
    - As our project mainly focuses on the "Data Analyst & Business Analyst" roles, we choose those

In [22]:
data_analyst_roles = [
    "Data Analyst",
    "Senior Data Analyst",
    "Lead Data Analyst",
    "Business Analyst"
]

- We define the final columns to be used to narrow down the important data for analysis

In [23]:
final_columns = [
    "job_role",
    "country",
    "is_remote",
    "schedule_type",
    "posted_date",
    "salary_year_avg",
    "no_degree_required",
    "health_insurance",
    "sql",
    "python",
    "excel",
    "tableau",
    "r",
    "power bi",
    "job_skills_clean"
]


- There are timestamps included in the posted_date column, where the time of the posting has no analysis value, we narrow it only to the date

In [24]:
df["posted_date"] = pd.to_datetime(df["posted_date"])
df["posted_date"] = df["posted_date"].dt.date

- There are multiple types of skills in the job_skills
- As we know the core skills of the role, we focus on the core skills
- We also create seperate columns for each skill in which if the job listing is required a specific skill, we use 0 & 1 for marking the requirement

In [25]:
df.head()

,job_role,job_location,schedule_type,is_remote,posted_date,no_degree_required,health_insurance,country,salary_year_avg,company_name,job_skills,job_type_skills
0,Data Scientist,"San José Province, San José, Costa Rica",Full-time,False,2023-08-01,False,False,Costa Rica,109500.0,Netskope,"['gdpr', 'excel']","{'analyst_tools': ['excel'], 'libraries': ['gd..."
1,Data Engineer,"Arlington, VA",Full-time,False,2023-06-26,False,False,United Kingdom,140000.0,Intelletec,"['mongodb', 'mongodb', 'python', 'r', 'sql', '...","{'analyst_tools': ['tableau'], 'cloud': ['orac..."
2,Data Engineer,Anywhere,Full-time,True,2023-02-21,False,True,United States,120000.0,Apex Systems,"['sql', 'python']","{'programming': ['sql', 'python']}"
3,Data Scientist,"Mountain View, CA",Full-time,False,2023-07-31,False,True,United States,228222.0,TikTok,"['sql', 'r', 'python', 'express']","{'programming': ['sql', 'r', 'python'], 'webfr..."
4,Data Analyst,Anywhere,Full-time,True,2023-10-12,False,True,United States,89000.0,Get It Recruit - Transportation,"['python', 'r', 'alteryx', 'tableau']","{'analyst_tools': ['alteryx', 'tableau'], 'pro..."


In [26]:
import ast

# Parse job_skills from string to list
df["job_skills_parsed"] = df["job_skills"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else []
)

# Normalize skills (lowercase, stripped)
df["job_skills_clean"] = df["job_skills_parsed"].apply(
    lambda skills: [s.strip().lower() for s in skills]
)

# Define core skills
core_skills = ["sql", "excel", "python", "tableau", "r", "power bi"]

# Create binary skill columns
for skill in core_skills:
    df[skill] = df["job_skills_clean"].apply(
        lambda skills: 1 if skill in skills else 0
    )

df_data_analyst = df[
    df["job_role"].isin([
        "Data Analyst",
        "Senior Data Analyst",
        "Lead Data Analyst",
        "Business Analyst"
    ])
][final_columns].copy()


- We check the job roles to validate the normalizations

In [27]:
df_data_analyst.shape

df_data_analyst["job_role"].value_counts()


job_role
Data Analyst           4772
Senior Data Analyst    1009
Business Analyst        497
Name: count, dtype: int64

- We then save the changes to a new CSV file for cleaner access during analysis

In [28]:
bool_cols = [
    "sql","python","excel","tableau","r","power bi",
    "no_degree_required","is_remote","health_insurance"
]

for col in bool_cols:
    df[col] = df[col].replace({"True": 1, "False": 0}).astype(bool)


In [29]:
df_data_analyst_save = df_data_analyst.copy()

df_data_analyst_save.to_csv("D:\Project\Job-Market-Intelligence\data\processed\data_analyst_jobs_final.csv", index=False)


- We then import the newly created dataset and check the skills column whether the skills listed are realistically true and find the top skills
- This also gives us the insight on the top job skills required and validate our core skills definition

In [30]:

df = pd.read_csv("D:\Project\Job-Market-Intelligence\data\processed\data_analyst_jobs_final.csv")

In [31]:
df["sql"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 6278 entries, 0 to 6277
Series name: sql
Non-Null Count  Dtype
--------------  -----
6278 non-null   int64
dtypes: int64(1)
memory usage: 49.2 KB


In [32]:
import ast


df['job_skills_clean'] = df['job_skills_clean'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])

df_exploded = df.explode('job_skills_clean')

print(df_exploded['job_skills_clean'].value_counts().head(20))

job_skills_clean
sql           4129
excel         2637
python        2397
tableau       2317
r             1389
power bi      1387
sas           1312
powerpoint     668
word           640
oracle         449
sql server     441
looker         404
aws            394
azure          393
snowflake      391
go             390
flow           379
spss           260
vba            244
sap            238
Name: count, dtype: int64


----------

- The data cleaning and normalization of the dataset is over
- The dataset is ready to be used in further analysis using SQL and create dashboards
- Key columns are neatly preserved

----------

